In [1]:
from multiprocessing import Process, Manager, Pool
import traitlets
from traitlets import HasTraits, Bool, observe
import time
from memory_profiler import profile

# Here we're going to try and map an old controller via wireless for control
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

from jetcam.csi_camera import CSICamera
import datetime
import numpy as np
from IPython.display import display
import ipywidgets
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input

In [2]:
input_image = Input(shape=(224,224,3), name='image')
conv1 = Conv2D(24, (5,5), name='conv1', strides=(2,2), padding='valid', activation='relu', kernel_initializer='he_normal')(input_image)
conv2 = Conv2D(36, (5,5), name='conv2', strides=(2,2), padding='valid', activation='relu', kernel_initializer='he_normal')(conv1)
conv3 = Conv2D(48, (5,5), name='conv3', strides=(2,2), padding='valid', activation='relu', kernel_initializer='he_normal')(conv2)

dropout_1 = Dropout(0.5)(conv3)

conv4 = Conv2D(64, (3,3), name='conv4', strides=(1,1), padding='valid', activation='relu', kernel_initializer='he_normal')(dropout_1)
conv5 = Conv2D(64, (3,3), name='conv5', strides=(1,1), padding='valid', activation='relu', kernel_initializer='he_normal')(conv4)

flatten = Flatten(name='flatten')(conv5)

# input_acceleration = Input(shape=(1,), name='acceleration_input')
# acceleration = Dense(1, name='acceleration_layer', activation='tanh', kernel_initializer='he_normal')(input_acceleration)

# concat = concatenate([flatten, acceleration])

fc1 = Dense(100, name='fc1', activation='relu', kernel_initializer='he_normal')(flatten)
fc2 = Dense(50, name='fc2', activation='relu', kernel_initializer='he_normal')(fc1)
fc3 = Dense(10, name='fc3', activation='relu', kernel_initializer='he_normal')(fc2)
output_steering = Dense(1, name='steering_output', activation='tanh', kernel_initializer='he_normal')(fc3)
#output_acceleration = Dense(1, name='acceleration_output', activation='tanh', kernel_initializer='he_normal')(fc3)

model = tf.keras.Model(inputs=input_image, outputs=output_steering)

steering_output_target = tf.placeholder(name="steering_output_target", dtype=tf.float32)

# model.compile(loss='mse', optimizer='Adam', metrics=['accuracy'])
model.compile(loss='mse', optimizer='Adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
model.load_weights('/home/jetson/jetracer/notebooks/latest_model/22-10')

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 110, 110, 24)      1824      
_________________________________________________________________
conv2 (Conv2D)               (None, 53, 53, 36)        21636     
_________________________________________________________________
conv3 (Conv2D)               (None, 25, 25, 48)        43248     
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 48)        0         
_________________________________________________________________
conv4 (Conv2D)               (None, 23, 23, 64)        27712     
_________________________________________________________________
conv5 (Conv2D)               (None, 21, 21, 64)        36928     
__________

In [5]:
camera = CSICamera(width=224, height=224, capture_width=1080, capture_height=720, capture_fps=20)
image = ipywidgets.widgets.Image(format='jpeg', width=224, height=224)
steering_slider = ipywidgets.widgets.FloatSlider(description="Steering Value", min=-1.0, max=1.0, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
#camera.running = True

#controller_dicts = []

def _ndarray_feature(ndarray: np.ndarray):
    assert isinstance(ndarray, np.ndarray) # check if ndarray
    dtype = ndarray.dtype
    if dtype == np.float64 or dtype == np.float32:
        return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
    elif dtype == np.int64:
        return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
    else:
        raise ValueError(f"Input should be a numpy ndarray. Instead got {ndarray.dtype}")

display(ipywidgets.HBox([image, steering_slider]))

In [ ]:
graph = tf.get_default_graph()

car.throttle = 0.25

while True:
    image = camera.read()
    image = image.reshape(1,224,224,3)
    output = model.predict(image)
    steering_slider.value = output
    steer = float(output[0])
    car.steering = steer